In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import torch


/home/melk/miniconda3/envs/WLS_CPU_python/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-28 00:48:30.434310: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 00:48:30.580932: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-28 00:48:30.580960: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have

In [2]:
sentiment_map = {0: "negative", 1: "positive"}
emotion_map = {0: "anger", 1: "fear", 2 : "joy", 3: "sadness"}
download = False
save_model_locally= False
if download:
    tokenizer_s = AutoTokenizer.from_pretrained("MilaNLProc/feel-it-italian-sentiment", cache_dir="data/")
    model_s = AutoModelForSequenceClassification.from_pretrained("MilaNLProc/feel-it-italian-sentiment", cache_dir="data/")
    model_s.eval()
    tokenizer_emo = AutoTokenizer.from_pretrained("MilaNLProc/feel-it-italian-emotion", cache_dir="data/")
    model_emo = AutoModelForSequenceClassification.from_pretrained("MilaNLProc/feel-it-italian-emotion", cache_dir="data/")
    model_emo.eval()
    if save_model_locally:
        model.save_pretrained('./local_models/sentiment_ITA')
        tokenizer.save_pretrained('./local_models/sentiment_ITA')
        model_emo.save_pretrained('./local_models/emotion_ITA')
        tokenizer_emo.save_pretrained('./local_models/emotion_ITA')
else:
    tokenizer_s = AutoTokenizer.from_pretrained("./local_models/sentiment_ITA/")
    model_s = AutoModelForSequenceClassification.from_pretrained("./local_models/sentiment_ITA/", num_labels=2)
    model_s.eval()
    tokenizer_emo = AutoTokenizer.from_pretrained("./local_models/emotion_ITA/")
    model_emo = AutoModelForSequenceClassification.from_pretrained("./local_models/emotion_ITA/", num_labels=4)
    model_emo.eval()

In [3]:
from transformers import pipeline
import re

generator = pipeline(task="text-classification", model=model_s, tokenizer=tokenizer_s, return_all_scores =True)
generator_emo = pipeline(task="text-classification", model=model_emo, tokenizer=tokenizer_emo, return_all_scores =True)

def sentiment_emoji(input_abs):

    if(input_abs ==""):
        return "🤷‍♂️"
        
    res = generator(input_abs)[0]
    res = {res[x]["label"]: res[x]["score"] for x in range(len(res))}
    res["🙂"] = res.pop("positive")
    res["🙁"] = res.pop("negative")
    return res


def emotion_emoji(input_abs):
    if(input_abs ==""):
        return "🤷‍♂️"

    res = generator_emo(input_abs)[0]
    res = {res[x]["label"]: res[x]["score"] for x in range(len(res))}
    res["😃"] = res.pop("joy")
    res["😡"] = res.pop("anger")
    res["😨"] = res.pop("fear")
    res["😟"] = res.pop("sadness")
   
    return res

/home/melk/miniconda3/envs/WLS_CPU_python/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
import mlflow.pyfunc
model_path_sentiment = "NLP_model_sentiment"
model_path_emotions = "NLP_model_emotions"

# Define the model class
class NPL_model_sentiment(mlflow.pyfunc.PythonModel):
    def __init__(self, in_string):
        self.input = in_string

    def predict(self, context, model_input):
        generator = pipeline(task="text-classification", model=model_s, tokenizer=tokenizer_s, return_all_scores =True)
        return generator(model_input)


# Define the model class
class NPL_model_emotions(mlflow.pyfunc.PythonModel):
    def __init__(self, in_string):
        self.input = in_string

    def predict(self, context, model_input):
        generator_emo = pipeline(task="text-classification", model=model_emo, tokenizer=tokenizer_emo, return_all_scores =True)
        return generator_emo(model_input)

In [5]:
#mlflow.end_run()
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [6]:
mlflow.set_experiment('text-classification-sentiment')
with mlflow.start_run():
    mlflow.pyfunc.log_model(artifact_path=model_path_sentiment, 
                    loader_module=None, 
                    data_path=None, 
                    code_path=None,
                    python_model=NPL_model_sentiment(in_string = "sono molto felice"),
                    registered_model_name="Sentiment"
                    )

mlflow.set_experiment('text-classification-emotions')
with mlflow.start_run():
    mlflow.pyfunc.log_model(artifact_path=model_path_emotions, 
                    loader_module=None, 
                    data_path=None, 
                    code_path=None,
                    python_model=NPL_model_emotions(in_string = " Io ti estirperò Saruman, come il veleno viene estirpato da una ferita… Saruman: Vai via tu, io sono l'ospite!"),
                    registered_model_name="Emotions"
                    )

2023/01/28 00:48:52 WARNING mlflow.utils.requirements_utils: Found torch version (1.13.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==1.13.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Registered model 'Sentiment' already exists. Creating a new version of this model...
2023/01/28 00:48:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Sentiment, version 3
Created version '3' of model 'Sentiment'.
2023/01/28 00:49:03 WARNING mlflow.utils.requirements_utils: Found torch version (1.13.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==1.13.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local versi